In [1]:
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Dense, Input
from keras.datasets import mnist

import utils

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
from PIL import Image  

def img2array(path):
    img = Image.open(path).convert('F')
    return np.array(img).ravel()

In [ ]:
# Размер батча данных и количество эпох
batch_size = 8 #128
nb_epoch = 10
files_count = 100#1491

In [ ]:
%%time
train_path = 'data/train/{}.jpg'
train_files = [train_path.format(x) for x in range(1,files_count+1)]
train = np.array([img2array(path) for path in train_files])
print(train.shape)

In [ ]:
%%time
train_mask_path = 'data/train_mask/{}.png'
train_mask_files = [train_mask_path.format(x) for x in range(1,files_count+1)]
train_mask = np.array([img2array(path) for path in train_mask_files])
print(train_mask.shape)

In [ ]:
%%time
test_path = 'data/test/{}.jpg'
test_files = [test_path.format(x) for x in range(1492,1492+100)]
test = np.array([img2array(path) for path in test_files])
print(test.shape)

In [ ]:
x_train = train.astype('float32') / 255.
y_train = train_mask.astype('float32') / 255.
x_test = test.astype('float32') /255.

In [ ]:
# Преобразуем картинки в вектора 
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
y_train = y_train.reshape((len(y_train), np.prod(y_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape)

In [ ]:
# Соберем модель автоэнкодера
input_img = Input(shape=(x_train.shape[1],))

# Размер скрытого слоя
nb_hidden = 500
encoded = Dense(nb_hidden, activation='relu')(input_img)

decoded = Dense(x_train.shape[1], activation='sigmoid')(encoded)

autoencoder = Model(inputs=input_img, outputs=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.summary()

In [ ]:
autoencoder.fit(x_train, y_train,
                epochs=nb_epoch, 
                batch_size=batch_size, 
                shuffle=True, 
                verbose=1)#,
                #validation_data=(x_test_noisy, x_test))

In [ ]:
# Decode test images 
decoded_imgs = autoencoder.predict(x_test)

In [ ]:
n = 10  # how many digits we will display
plt.figure(figsize=(20, 6))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_train[i].reshape(320, 240))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display original with noise
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(y_train[i].reshape(320, 240))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(decoded_imgs[i].reshape(320, 240))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## На картинке:
## 1 ряд - оригинальные изображения
## 2 ряд - маски
## 3 ряд - маски энкодера

------

In [ ]:
predictions = []
for dec_im in decoded_imgs:
    predictions.append(dec_im[i].reshape(320, 240))
    

In [ ]:
# save result
df = pd.DataFrame.from_dict({'image': path_images, 'rle_mask': predictions})
df.to_csv('baseline_submission.csv', index=False)